In [1]:
# Définir les paramètres de la courbe de Montgomery (Curve25519)
p = 2^255 - 19
A = 486662
F = GF(p)

# Définir un point de base P avec seulement la coordonnée x
xP = F(9)

def xADD(XP, ZP, XQ, ZQ, X_diff, p):
    F = GF(p)
    A = F(XP + ZP)
    B = F(XP - ZP)
    C = F(XQ + ZQ)
    D = F(XQ - ZQ)
    DA = D * A
    CB = C * B
    X_out = (DA + CB)^2
    Z_out = X_diff * (DA - CB)^2
    return X_out, Z_out



def xDBL(XP, ZP, a24, p):
    F = GF(p)
    A = F(XP + ZP)
    B = F(XP - ZP)
    AA = A^2
    BB = B^2
    E = AA - BB
    X_out = AA * BB
    Z_out = E * (BB + a24 * E)
    return X_out, Z_out


def montgomery_ladder(xP, A, n, p):
    F = GF(p)
    a24 = F((A + 2) / 4)

    # Représentation projective : (X : Z)
    X0, Z0 = F(1), F(0)      # Point à l'infini (0*P)
    X1, Z1 = F(xP), F(1)     # Point P

    n_bin = bin(n)[2:]       # Bits de n sans '0b'

    for bit in n_bin:
        bit = int(bit)
        if bit == 0:
            # P = P, Q = Q
            X1, Z1 = xADD(X0, Z0, X1, Z1, xP, p)
            X0, Z0 = xDBL(X0, Z0, a24, p)
        else:
            # P = Q, Q = P
            X0, Z0 = xADD(X1, Z1, X0, Z0, xP, p)
            X1, Z1 = xDBL(X1, Z1, a24, p)

    return X0 / Z0


In [20]:
# Corps fini et courbe de Montgomery : By^2 = x(x^2 + Ax + 1)
K = GF(101)
A = K(3)
#B = K(1)

E = EllipticCurve(K, [0, A, 0, 1, 0])  # y^2 = x(x^2 + Ax + 1)

R.<X> = PolynomialRing(K)


def kummer_poly(xP, xQ, A):
    trace = -2*((xP*xQ + 1)*(xP + xQ) + 2*A*xP*xQ) / ((xP - xQ)^2)
    product = ((xP*xQ - 1)^2) / ((xP - xQ)^2)
    return X^2 + trace*X + product

# Choix des points
T= E.random_point()

while True:
    P = E.random_point()
    Q = E.random_point()
    if P != -Q and P != Q and P != E(0):
        break

xP = P[0]
xQ = Q[0]


In [30]:
def pgcd_polynomes(f, g, verbose=False):
    """
    Calcule manuellement le PGCD de deux polynômes f et g
    en utilisant l'algorithme d'Euclide, sans utiliser .gcd().
    
    Entrées :
        - f, g : polynômes dans un même anneau (par exemple R.<X> = PolynomialRing(QQ))
        - verbose (optionnel) : si True, affiche les étapes de l'algorithme
        
    Sortie :
        - Le PGCD de f et g
    """
    
    if f.parent() != g.parent():
        raise ValueError("Les deux polynômes doivent appartenir au même anneau.")
    
    while not g.is_zero():
        q, r = f.quo_rem(g)  # Division euclidienne
        if verbose:
            print(f"Division : {f} = ({q}) * ({g}) + ({r})")
        f, g = g, r  # Mettre à jour
        
    # À la fin, f est le PGCD
    return f.monic()  # Optionnel : rendre le PGCD monique (coefficient dominant 1)


T= E.random_point()

while True:
    P = E.random_point()
    Q = E.random_point()
    if P != -Q and P != Q and P != E(0):
        break
f=kummer_poly(P[0], (Q+T)[0], A)
g=kummer_poly(Q[0], (P+T)[0], A)
print(f)
print(g)
pgcd_polynomes(f, g, verbose=True)

def direct_poly(P, Q):
    return (X - (P + Q)[0]) * (X - (P - Q)[0])
f2=direct_poly(P, Q+T)
g2=direct_poly(Q, P+T)

X^2 + 28*X + 95
X^2 + 91*X + 68
Division : X^2 + 28*X + 95 = (1) * (X^2 + 91*X + 68) + (38*X + 27)
Division : X^2 + 91*X + 68 = (8*X + 10) * (38*X + 27) + (0)


In [22]:
T


(61 : 59 : 1)

In [23]:
f

X^2 + 88*X + 1

In [24]:
f.parent()

Univariate Polynomial Ring in X over Finite Field of size 101

In [25]:
f.gcd(g)


1

In [26]:
f.roots()

[(61, 1), (53, 1)]

In [27]:
g.roots()

[(94, 1), (83, 1)]

In [28]:
P+Q+T

(24 : 27 : 1)

In [29]:
f2

X^2 + 98*X + 1

In [16]:
f2.gcd(g2)

X + 11

In [17]:
P

(49 : 41 : 1)

In [18]:
P.parent()

Abelian group of points on Elliptic Curve defined by y^2 = x^3 + 3*x^2 + x over Finite Field of size 101

In [19]:
A

3